In [1]:
import numpy as np
import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report
# from imblearn.over_sampling import SMOTE
# from sklearn.feature_selection import VarianceThreshold
from scipy.stats import pearsonr
import random
import tensorflow as tf
import pandas as pd
import csv
import math

In [2]:
file_path = "CCHIT.csv"
save_path = "IR_dataset/IR_CCHIT.csv"

In [3]:
def read_csv_as_list(file_path):
    data = []
    with open(file_path, 'r', newline='', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        first_row = True
        for row in csv_reader:
            if first_row:  
                first_row = False
                continue
            data.append(row)
    return data

In [4]:
csv_data = read_csv_as_list(file_path)

In [5]:
high_column = list({row[0] for row in csv_data})
low_column = list({row[1] for row in csv_data})

## TF-IDF

In [9]:
def vsm_count_key(text):
    words = text.split()
    table = {}
    for word in words:
        if word != "":
            if word in table:
                table[word] += 1
            else:
                table[word] = 1
    return sorted(table.items(), key=lambda item: item[1], reverse=True)


def vsm_merge_keys(dic1, dic2):
    arrayKey = []
    for item in dic1:
        arrayKey.append(item[0])
    for item in dic2:
        if item[0] not in arrayKey:
            arrayKey.append(item[0])
    
    arrayNum1 = [0] * len(arrayKey)
    arrayNum2 = [0] * len(arrayKey)
    
    for i, key in enumerate(arrayKey):
        for k, v in dic1:
            if key == k:
                arrayNum1[i] = v
        for k, v in dic2:
            if key == k:
                arrayNum2[i] = v

    dot_product = sum(a * b for a, b in zip(arrayNum1, arrayNum2))
    magnitude1 = math.sqrt(sum(a * a for a in arrayNum1))
    magnitude2 = math.sqrt(sum(b * b for b in arrayNum2))
    if magnitude1 == 0 or magnitude2 == 0:
        return 0
    return dot_product / (magnitude1 * magnitude2)

def compute_similarity(text1, text2):

    dic1 = vsm_count_key(text1)
    dic2 = vsm_count_key(text2)

    result = vsm_merge_keys(dic1, dic2)
    return result

In [10]:
csv_data[0][0]

'The system shall allow a user to foward notification to someone else at the site'

In [14]:
tfidf_dataset = []
for i in range(len(csv_data)):
    text1 = csv_data[i][0]
    text2 = csv_data[i][1]
    result = compute_similarity(text1, text2)
    tfidf_dataset.append(result)

# print(tfidf_dataset)

In [15]:
len(tfidf_dataset)

252

## Okapi BM25

In [16]:
import math
from collections import Counter

def compute_idf(docs):
    D = len(docs)
    df = {}
    for doc in docs:
        for word in set(doc):
            df[word] = df.get(word, 0) + 1
    
    idf = {}
    for word, freq in df.items():
        idf[word] = math.log((D - freq + 0.5) / (freq + 0.5) + 1)
    
    return idf

def compute_score(doc1, doc2, idf):
    avgdl = (len(doc1) + len(doc2)) / 2
    k1 = 1.5
    b = 0.75
    score = 0
    doc1_counter = Counter(doc1)
    doc2_counter = Counter(doc2)
    for word in set(doc1 + doc2):
        if word not in idf:
            continue
        idf_val = idf[word]
        tf1 = doc1_counter[word]
        tf2 = doc2_counter[word]
        doc1_len = len(doc1)
        doc2_len = len(doc2)
        score += (idf_val * ((tf1 * (k1 + 1)) / (tf1 + k1 * (1 - b + b * doc1_len / avgdl))) *
                              ((tf2 * (k1 + 1)) / (tf2 + k1 * (1 - b + b * doc2_len / avgdl))))
    return score

def compute_bm25_similarity(doc1, doc2):
    idf = compute_idf([doc1, doc2])
    score = compute_score(doc1, doc2, idf)
    return score


In [17]:
BM25_dataset = []
for i in range(len(csv_data)):
    text1 = csv_data[i][0]
    text2 = csv_data[i][1]
    result = compute_bm25_similarity(text1, text2)
    BM25_dataset.append(result)

In [18]:
BM25_dataset

[10.261973224867328,
 11.117247995112578,
 8.92132304247616,
 9.435237413878538,
 10.179668920536955,
 11.770119223844231,
 10.776218185874926,
 12.87633104840782,
 11.234692953218097,
 13.77907790462664,
 9.893233108486474,
 11.938213560823142,
 12.470807174483172,
 10.29327931419529,
 11.16533849510064,
 8.25131761803893,
 10.317576391126572,
 5.514574661582996,
 9.02465251926342,
 9.343025369793114,
 10.381412703137178,
 9.719739306197022,
 8.330380091326347,
 12.760185441985012,
 12.318475071863237,
 13.868302118090623,
 7.805861588815944,
 9.977986640991771,
 10.465548644626411,
 12.062766068900348,
 10.842160602460178,
 10.54105542501929,
 13.111962448135879,
 11.653010728469319,
 9.223411787734813,
 9.15889356663311,
 10.48066789969187,
 10.750733623016487,
 9.560115831942618,
 9.92345992176589,
 10.113779949934921,
 11.470401873779645,
 10.651242030629597,
 11.513133513995259,
 11.243838216534103,
 12.219092008512279,
 10.248771015953466,
 9.57753051608943,
 13.451181285199485,

## Jensen Shannon

In [20]:
import numpy as np
from scipy.spatial.distance import jensenshannon
from collections import Counter


def count_words(text):
    words = text.split()
    return Counter(words)

def compute_js_similarity(text1, text2):
    counter1 = count_words(text1)
    counter2 = count_words(text2)

    all_words = list(set(counter1.keys()).union(set(counter2.keys())))

    vector1 = np.array([counter1.get(word, 0) for word in all_words], dtype=float)
    vector2 = np.array([counter2.get(word, 0) for word in all_words], dtype=float)

    vector1 /= vector1.sum()
    vector2 /= vector2.sum()
    
    js_distance = jensenshannon(vector1, vector2)
    js_similarity = 1 - js_distance
    return js_similarity



In [22]:
Jensen_Shannon_dataset = []
for i in range(len(csv_data)):
    text1 = csv_data[i][0]
    text2 = csv_data[i][1]
    result = compute_js_similarity(text1, text2)
    Jensen_Shannon_dataset.append(result)

In [23]:
Jensen_Shannon_dataset

[0.4261900807736465,
 0.326230547940511,
 0.3362903799211808,
 0.32022200655412736,
 0.3930927254655858,
 0.3543487912523329,
 0.30790105186628725,
 0.2650750786541355,
 0.3032613031624798,
 0.3169310406882899,
 0.33029334840853874,
 0.33103096988758296,
 0.33739693039213936,
 0.3857624948041001,
 0.3553412839891944,
 0.2012741582255282,
 0.2601833334841108,
 0.1674453888423022,
 0.20484203002461543,
 0.26334276608598395,
 0.30636038189380943,
 0.1941042012070664,
 0.320811036265791,
 0.37400399327018274,
 0.30328672969093573,
 0.29779285073090866,
 0.3803109884053536,
 0.3866205072425636,
 0.34805814461048257,
 0.3652072503200373,
 0.34825636028823137,
 0.25726594084430465,
 0.3272034283002051,
 0.28962678398139097,
 0.29645196927295103,
 0.27993230624427,
 0.284131640619551,
 0.2678020828059592,
 0.29030313514933426,
 0.2799323062442699,
 0.2714797118670147,
 0.284131640619551,
 0.29134427958881093,
 0.30497610087556726,
 0.327603627868151,
 0.31715378993729937,
 0.32216331320921476,

## Latent Semantic Analysis (LSA)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def cosine_lsa_similarity_measure(text1, text2, n=1):
    
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform([text1, text2])
    
    
    svd_model = TruncatedSVD(n_components=n)  
    X2 = svd_model.fit_transform(X)  
    cosine_sim = cosine_similarity(X)
    return cosine_sim[0, 1]


In [26]:
lsa_dataset = []
for i in range(len(csv_data)):
    text1 = csv_data[i][0]
    text2 = csv_data[i][1]
    result = cosine_lsa_similarity_measure(text1, text2,1)
    lsa_dataset.append(result)

In [28]:
len(lsa_dataset)

252

## Latent Dirichlet Allocation (LDA)

In [30]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from gensim.corpora import Dictionary
from gensim.models import LdaModel

# Download necessary resources
nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    """ Preprocess text: lowercase, tokenize, remove stopwords. """
    stop_words = set(stopwords.words('english'))
    words = [w.lower() for w in nltk.word_tokenize(text) if w.lower() not in stop_words]
    return words

def lda_similarity(text1, text2):
    """ Calculate LDA similarity between two text strings. """
    processed_text1 = preprocess_text(text1)
    processed_text2 = preprocess_text(text2)
    
    # Create dictionary and corpus
    dictionary = Dictionary([processed_text1, processed_text2])
    corpus = [dictionary.doc2bow(processed_text1), dictionary.doc2bow(processed_text2)]
    
    # Train LDA model
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=2)
    
    # Get document topics
    doc_bow1 = dictionary.doc2bow(processed_text1)
    doc_bow2 = dictionary.doc2bow(processed_text2)
    
    doc_lda1 = lda_model.get_document_topics(doc_bow1)
    doc_lda2 = lda_model.get_document_topics(doc_bow2)
    
    # Create dense topic distribution vectors
    vec1 = np.zeros(lda_model.num_topics)
    vec2 = np.zeros(lda_model.num_topics)
    for topic, prob in doc_lda1:
        vec1[topic] = prob
    for topic, prob in doc_lda2:
        vec2[topic] = prob
    
    # Compute cosine similarity
    if np.all(vec1 == 0) or np.all(vec2 == 0):
        return 0
    similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    return similarity

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
lda_dataset = []
for i in range(len(csv_data)):
    text1 = csv_data[i][0]
    text2 = csv_data[i][1]
    result = lda_similarity(text1, text2)
    lda_dataset.append(result)

In [34]:
len(lda_dataset)

252

## Dirichlet 

In [35]:
import nltk
from nltk.probability import FreqDist, MLEProbDist
from nltk.corpus import stopwords
import numpy as np
import math

nltk.download('stopwords')
nltk.download('punkt')

class DirichletLanguageModel:
    def __init__(self, documents, alpha=0.1):
        self.alpha = alpha  # Dirichlet prior
        self.token_counts = FreqDist()  # Total count of tokens
        self.doc_lengths = []  # Length of each document
        self.vocab_size = 0  # Size of the vocabulary
        self.build_model(documents)

    def build_model(self, documents):
        for doc in documents:
            tokens = nltk.word_tokenize(doc.lower())
            filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]
            self.doc_lengths.append(len(filtered_tokens))
            self.token_counts.update(filtered_tokens)
        self.vocab_size = len(self.token_counts)

    def get_probability(self, term):
        """ Calculate probability of the term with Dirichlet smoothing """
        term_count = self.token_counts[term]
        total_tokens = sum(self.doc_lengths)
        return (term_count + self.alpha) / (total_tokens + self.alpha * self.vocab_size)

    def get_document_probability(self, doc):
        """ Calculate the probability of a document """
        prob = 0
        doc_tokens = nltk.word_tokenize(doc.lower())
        for token in doc_tokens:
            prob += math.log(self.get_probability(token))
        return prob


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
Dirichlet_dataset = []
for i in range(len(csv_data)):
    text1 = csv_data[i][0]
    text2 = csv_data[i][1]
    documents = [text1,text2]
    model = DirichletLanguageModel(documents)
    doc_to_test = text1
    result = model.get_document_probability(doc_to_test)
    Dirichlet_dataset.append(result)

In [42]:
len(Dirichlet_dataset)

252

## Jelenik-Mercer

In [43]:
import numpy as np

class JelinekMercerSmoothing:
    def __init__(self, lambda_=0.5):
        self.lambda_ = lambda_
        self.unigram_counts = {}
        self.bigram_counts = {}

    def calculate_unigram_counts(self, text):
        unigram_counts = {}
        for word in text.split():
            unigram_counts[word] = unigram_counts.get(word, 0) + 1
        return unigram_counts

    def calculate_bigram_counts(self, text):
        bigram_counts = {}
        tokens = text.split()
        for i in range(1, len(tokens)):
            bigram = (tokens[i-1], tokens[i])
            bigram_counts[bigram] = bigram_counts.get(bigram, 0) + 1
        return bigram_counts

    def unigram_probability(self, word):
        return self.unigram_counts.get(word, 0) / sum(self.unigram_counts.values())

    def bigram_probability(self, word, prev_word):
        numerator = self.bigram_counts.get((prev_word, word), 0)
        denominator = self.unigram_counts.get(prev_word, 0)
        if denominator == 0:
            return 0
        else:
            return numerator / denominator

    def jelinek_mercer_probability(self, word, prev_word):
        p_interp = self.lambda_ * self.bigram_probability(word, prev_word) + (1 - self.lambda_) * self.unigram_probability(word)
        return p_interp

    def text_similarity(self, text1, text2):
        self.unigram_counts = self.calculate_unigram_counts(text1 + " " + text2)
        self.bigram_counts = self.calculate_bigram_counts(text1 + " " + text2)
        
        tokens1 = text1.split()
        tokens2 = text2.split()
        
        # Calculate similarity based on bigram probabilities
        similarity = 1.0
        for i in range(1, len(tokens1)):
            prev_word = tokens1[i-1]
            word = tokens1[i]
            if prev_word in tokens2 and word in tokens2:
                sim_word = min(self.jelinek_mercer_probability(word, prev_word), self.jelinek_mercer_probability(prev_word, word))
                similarity *= sim_word
        
        return similarity

# Example usage
jm_smoothing = JelinekMercerSmoothing()

# text1 = "the quick brown fox"
# text2 = "jumps over the lazy dog"

# similarity = jm_smoothing.text_similarity(text1, text2)
# print("Similarity between text1 and text2:", similarity)


In [44]:
jm_smoothing_dataset = []
for i in range(len(csv_data)):
    text1 = csv_data[i][0]
    text2 = csv_data[i][1]
    result = jm_smoothing.text_similarity(text1, text2)
    jm_smoothing_dataset.append(result)

In [45]:
jm_smoothing_dataset

[9.5367431640625e-07,
 0.0006925207756232686,
 0.002066115702479339,
 0.001736111111111111,
 5.689576695493856e-05,
 3.0517578125e-05,
 3.498542274052478e-05,
 0.00041649312786339016,
 0.0007716049382716049,
 7.687892082496206e-06,
 0.0008650519031141869,
 0.0006510416666666666,
 2.0246193715581467e-05,
 1.2648397321575387e-06,
 0.00011104969103508183,
 1.0,
 0.0011890606420927466,
 1.0,
 1.0,
 0.0012755102040816326,
 0.0007304601899196495,
 1.0,
 0.0030864197530864196,
 7.224761580900885e-11,
 0.0004938271604938272,
 1.0973936899862826e-05,
 0.0030864197530864196,
 0.0011890606420927466,
 0.0008163265306122448,
 3.906250000000001e-07,
 0.0014792899408284025,
 0.0011111111111111111,
 0.00010337923129380014,
 1.0,
 0.0022675736961451243,
 0.001736111111111111,
 0.0009765625,
 0.0013717421124828531,
 0.002066115702479339,
 0.001736111111111111,
 0.0014792899408284025,
 3.0517578125e-05,
 3.7037037037037037e-05,
 0.0007716049382716049,
 0.0011890606420927466,
 0.0010405827263267429,
 0.00

In [46]:
merged_dataset = list(zip(tfidf_dataset, BM25_dataset, Jensen_Shannon_dataset,
                          lsa_dataset, lda_dataset, Dirichlet_dataset,
                          jm_smoothing_dataset))


merged_dataset = [list(item) for item in merged_dataset]

In [47]:
len(merged_dataset)

252

In [49]:
len(merged_dataset[0])

7

In [50]:
df = pd.DataFrame(merged_dataset)

In [52]:
df.to_csv(save_path, index=False, encoding='utf-8')